# Assignment 3 - SED Fitting

## Bayesian Statistics

This assignment is split into 3 sections, roughly corresponding to the contents of each of the 3 weeks in the Bayesian Statistics module. 

All assignments are presented as Jupyter notebooks. You will fork the repository to have your own access to all files. You can edit this notebook directly with your answers and push your changes to GitHub. 

### **The goal of this assignment is to use different MCMC and Bayesian inference techniques to fit SEDs to galaxy magnitudes**

# STEP 0 - Prospector Inputs

Prospector has some in-built MCMC techniques that you will have used in Assignment 1 (Emcee or Dynesty). For this assignment you will not be using these in-built tools, instead you will use external libraries to code your own MCMC results and perform model comparison on them. We will however still use prospector for the model so lets prepare that here

### The model we are using is a very simple parametric model with 6 free parameters

* ### $z$ redshift
* ### $M_{\rm star}$ stellar mass
* ### $\log(z_{\rm sol})$ metallicity?
* ### dust V-band optical depth
* ### $t_{\rm age}$ The age of the host galaxy
* ### $\tau$


1. Activate the enviroment/kernel you used with prospector installed

2. Prepare the prospector model (if you like you can edit the below model, but you dont have to)

In [21]:
from prospect.models import SedModel, priors
from prospect.models.templates import TemplateLibrary
from prospect.sources import CSPSpecBasis
import time

model_params = TemplateLibrary["parametric_sfh"]

# Let redshift vary
model_params["zred"]["isfree"] = True
model_params['zred']['init'] = 0.1
model_params['zred']['prior'] = priors.TopHat(mini=0,maxi=1)

# Build the model
prospector_model = SedModel(model_params)

sps = CSPSpecBasis(zcontinuous=1)


3. Load the data vector for a given galaxy (again you can change the gaalxy if you wish, maybe match one of your galaxies from assignment)

In [22]:
import sedpy 
import prospect
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table

gal_id = 33

with fits.open('../data/sw_input.fits') as f:
    df = Table(f[1].data).to_pandas()
    f.close()

def build_obs(gal_id):
    """Given an object, load in fluxes, convert them to nanomaggies, and create a dict used in Prospector."""

    inp = {}
    
    # Get dataframe row for the object
    row = df.iloc[gal_id]
    inp['redshift'] = row.redshift

    # Load the filter response curves from sedpy
    bands = [f'sdss_{filt}0' for filt in 'ugriz'] + [f'wise_w{n}' for n in range(1,5)]
    filters = sedpy.observate.load_filters(bands)
    inp['filters'] = filters
    
    # Fluxes and uncertainties - already in units of maggies
    cols = [f'flux_{filt}' for filt in 'ugriz'] + [f'flux_w{n}' for n in range(1,5)]
    fluxes = row[cols].values.astype(float) / 3631

    # Errors
    cols_err = [f'{col}_e' for col in cols]
    errs = row[cols_err].values.astype(float) / 3631

    # Anything with a value of 9.999 is null, so may need to mask those fluxes by editing phot_mask
    inp['maggies'] = fluxes
    inp['maggies_unc'] = errs
    inp['phot_mask'] = [True for val in fluxes] # Nothing masked here right now

    # This is an array of effective wavelengths for each of the filters.  
    # It is not necessary, but it can be useful for plotting so we store it here as a convenience
    inp["phot_wave"] = np.array([f.wave_effective for f in inp["filters"]])
    inp["wavelength"] = None
    
    # Populate other fields with default
    inp = prospect.utils.obsutils.fix_obs(inp)
    return inp

obs = build_obs(gal_id)

4. Prepare a function that takes the 6 parameters as input, and outputs the predicted fluxes in maggies (once this is set up, you shouldn't have to use prospector directly again for this assignment)

In [23]:
#a random sample from the prospectors default priors (just to make sure the model runs)
random_input_values = np.array([model_params[k]['prior'].sample()[0] for k in model_params.keys() if model_params[k]['isfree']==True])
param_names = np.array([model_params[k]['name'] for k in model_params.keys() if model_params[k]['isfree']==True])
print('Parameters should enter the function in this order:')
print(param_names)

### THIS IS THE FUNCTION YOU CAN USE FOR ALL 
### YOUR MODEL PREDICTIONS FROM THIS POINT
def model(theta):
    return prospector_model.predict(theta, obs=obs, sps=sps)[1]



Parameters should enter the function in this order:
['zred' 'mass' 'logzsol' 'dust2' 'tage' 'tau']


5. Prospector does some kind of caching under the hood such that the calls to the model that require uncached inputs take much longer than the cached ones

We can get around this by initially running the model at enough random parameter inputs to cover all of the possible cached file we will need (there is very likely a better way to do this)

Try to avoid calling the model with inputs outside of the range defined below

THIS LINE WILL TAKE A COUPLE MINUTES TO RUN


In [24]:
import scipy.stats
np.random.seed(1)

nsamples = 10

#prior boundaries
zred_min = 0.0
zred_max = 1.0
logmass_min = 8.0
logmass_max = 12.0
logzsol_min = -2.0
logzsol_max = 0.19
dust2_min = 0.0
dust2_max = 2.0
tage_min = 0.001
tage_max = 13.8
logtau_min = np.log10(0.1)
logtau_max = np.log10(30)
theta_min = np.array([zred_min, logmass_min, logzsol_min, dust2_min, tage_min, logtau_min])
theta_max = np.array([zred_max, logmass_max, logzsol_max, dust2_max, tage_max, logtau_max])

#generate some random input parameter combos
lhc_sampling = scipy.stats.qmc.LatinHypercube(6)
input_params = theta_min + lhc_sampling.random(nsamples) * (theta_max-theta_min)
#un-log the mass and tau 
input_params[:,1] = 10.**input_params[:,1]
input_params[:,5] = 10.**input_params[:,5]

model_evals = np.zeros( (nsamples, obs['ndof']) )
for i in range(nsamples):
    start = time.time()
    model_evals[i] = model(input_params[i])
    finish = time.time()
    print(i, f'{finish-start}s')

0 25.899236917495728s
1 26.49707317352295s
2 27.990087747573853s
3 12.903696775436401s
4 13.06036376953125s
5 13.435635089874268s
6 0.006163835525512695s
7 14.149084091186523s
8 0.005480051040649414s
9 0.00547480583190918s


# SECTION 1 - MCMC

1. Pick (and install) your 2 favourite MCMC sampling method/implementation you learned about in the lectures (At least 1 of them should be a "nested" sampling method, as we will use the features of nested sampling later)

Some examples are shown below

#### Metropolis-Hastings and Gibbs sampling 
* The most famous sampling methods 
* Many people code their own versions
* There are many examples and packages online to do this (some examples we haven't tested [here](https://github.com/WenjieZ/easyMH) [here](https://pymcmcstat.readthedocs.io/en/latest/#pymcmcstat) [here](https://numeryst.com/gibbs-sampling-an-introduction-with-python-implementation/#Code_Example))

#### Emsemble
* More sophisticated samplers 
* Affine-invariant [EMCEE](https://emcee.readthedocs.io/en/stable/)
* KDE [Kombine](https://github.com/bfarr/kombine)

#### Nested sampling 
* [Multinest](https://github.com/rjw57/MultiNest)
* [Polychord](https://github.com/PolyChord/PolyChordLite)
* [Dynesty](https://dynesty.readthedocs.io/en/latest/index.html) (Has some really great documentation)

#### Other 
* [Nautilus](https://nautilus-sampler.readthedocs.io/en/stable/)

In [25]:
### check they import here

2. Fit your model to your data using your 2 chosen sampling techniques

* Choose flat top-hat priors for all parameters (you can look at your results from Assignment 1 to get some reasonable ranges) 
* Plot the resulting parameter constraints on top of each other. e.g. in a corner plot
* Do they agree with each other?

In [26]:
### Space to work (Sampler 1)

In [ ]:
### Space to work (Sampler 2)

3. Did either of your sampling methods have a burn-in? if so make a plot showing the burn-in and justify how much burn-in to remove. If not, explain why there is no burn-in in your methods

In [27]:
### Space to work

# SECTION 2 - Model Comparison

1. Use the nested sampling technique you chose in Section 1 to compute the Baysian Evidence of your model fit

In [28]:
### Space to work

2. Change the model in some-way and re-run your nested sampling chain (e.g. add/remove a new parameter, or dramatically change the prior on a parameter)

In [29]:
### Space to work

3. Use a Bayesian model comparison technique to decide which model your data prefer

In [30]:
### Space to work

# SECTION 3 - Comparing results



1. How do the conclusions from your "Bayesian" model comparison compare to just looking at the change in the best-fit (reduced) chi-squared

In [31]:
### Space to work

2. How do your experiences of inferring galaxy properties from MCMC compare to your inference using Machine Learning (Assignment 2 - Section 3)

* Which one was more accurate?
* What are their limitations?
* What unique infomation did the different approaches provide?

In [32]:
### Space to work